In [1]:
import geopandas as gpd
import openeo
from openeo.extra.spectral_indices import compute_indices
import matplotlib.pyplot as plt
import numpy as np



In [3]:
# Define file path to the shapefile
shapefile_path = r"D:\Acres\December 2024\Assignment 1\Jharkhand\Dhanbad1.shp"

# Read the shapefile using GeoPandas
gdf = gpd.read_file(shapefile_path)

In [4]:
# Get the bounding box of the shapefile as the spatial extent
bounds = gdf.total_bounds
extent = {
    "west": bounds[0],
    "south": bounds[1],
    "east": bounds[2],
    "north": bounds[3],
}

In [5]:
# Connect to the OpenEO backend and authenticate
connection = openeo.connect("openeo.dataspace.copernicus.eu").authenticate_oidc()

# Define temporal extents for pre-fire and post-fire analysis
pre_fire_start = "2023-05-03"
pre_fire_end = "2023-08-03"
post_fire_start = "2023-08-04"
post_fire_end = "2023-11-03"

Authenticated using refresh token.


In [11]:
connection = openeo.connect("openeo.dataspace.copernicus.eu").authenticate_oidc()


Authenticated using refresh token.


In [6]:
# Load Sentinel-2 pre-fire collection
s2pre = connection.load_collection(
    "SENTINEL2_L2A",
    temporal_extent=[pre_fire_start, pre_fire_end],
    spatial_extent=extent,
    bands=["B04", "B08", "B12"],
    max_cloud_cover=90,
)

# Load Sentinel-2 post-fire collection
s2post = connection.load_collection(
    "SENTINEL2_L2A",
    temporal_extent=[post_fire_start, post_fire_end],
    spatial_extent=extent,
    bands=["B04", "B08", "B12"],
    max_cloud_cover=90,
)

In [12]:
# Compute Normalized Burn Ratio (NBR) for pre-fire and post-fire
def compute_nbr(collection):
    return collection.ndvi(nir="B08", red="B12")

nbr_pre = compute_nbr(s2pre)
nbr_post = compute_nbr(s2post)

# Compute delta Normalized Burn Ratio (dNBR)
dnbr = nbr_pre.subtract(nbr_post)

In [13]:
# Define output directory
output_dir = r"D:\JRF_IIT_BOMBAY\December\DNBR_CORPONICUS"

# Save results
nbr_pre_output = f"{output_dir}\nbr_pre.tif"
nbr_post_output = f"{output_dir}\nbr_post.tif"
dnbr_output = f"{output_dir}\dnbr.tif"

# Save results with correct method for saving
nbr_pre.download(nbr_pre_output, format="GTiff")
nbr_post.download(nbr_post_output, format="GTiff")
dnbr.download(dnbr_output, format="GTiff")

print(f"NBR pre-fire saved to {nbr_pre_output}")
print(f"NBR post-fire saved to {nbr_post_output}")
print(f"dNBR saved to {dnbr_output}")


<>:7: SyntaxWarning: invalid escape sequence '\d'
<>:7: SyntaxWarning: invalid escape sequence '\d'
C:\Users\Admin\AppData\Local\Temp\ipykernel_8652\855412932.py:7: SyntaxWarning: invalid escape sequence '\d'
  dnbr_output = f"{output_dir}\dnbr.tif"


OSError: [Errno 22] Invalid argument: 'D:\\JRF_IIT_BOMBAY\\December\\DNBR_CORPONICUS\nbr_pre.tif'

In [ ]:
# Generate and display images
plt.figure(figsize=(15, 5))

# Placeholder data for demonstration (replace with actual raster data when available)
nbr_pre_array = np.random.random((100, 100))  # Replace with actual data
nbr_post_array = np.random.random((100, 100))  # Replace with actual data
dnbr_array = nbr_pre_array - nbr_post_array  # Replace with actual computation

# Pre-fire NBR
plt.subplot(1, 3, 1)
plt.imshow(nbr_pre_array, cmap="RdYlGn")
plt.colorbar(label="NBR Value")
plt.title("Pre-Fire NBR")

# Post-fire NBR
plt.subplot(1, 3, 2)
plt.imshow(nbr_post_array, cmap="RdYlGn")
plt.colorbar(label="NBR Value")
plt.title("Post-Fire NBR")

# dNBR
plt.subplot(1, 3, 3)
plt.imshow(dnbr_array, cmap="RdYlGn")
plt.colorbar(label="dNBR Value")
plt.title("Delta NBR (dNBR)")

# Save the figure
image_output = f"{output_dir}\dnbr_analysis.png"
plt.savefig(image_output)
print(f"Analysis image saved to {image_output}")

# Calculate statistics
mean_pre = np.mean(nbr_pre_array)
mean_post = np.mean(nbr_post_array)
mean_dnbr = np.mean(dnbr_array)

print("Statistics:")
print(f"Mean Pre-Fire NBR: {mean_pre}")
print(f"Mean Post-Fire NBR: {mean_post}")
print(f"Mean dNBR: {mean_dnbr}")
